## The Zoning Problem 
The aim of this notebook is to generate the optimal zoning by formulating a linear assignment problem and using google
OR-Toolsfor solving the optimization problem.
The agent behaviours are taken from the previous study. The occupation and env lattices are random for the test problem .

### The Test Problem 
There are five agents :[Blue,Green,Yellow,Red,Violet]
The number of voxels which each agent occupies are [B1,G1,Y1,R1,V1]
Total number of voxels in a lattice = x 
Value lattice for the agents are =[Bv],[Gv],[Yv],[Rv],[Vv]

####  Aim 
: To find the best combination of Zones to achieve the maximum occupancy value
#### Steps

1. Generate the lattices , Agent behaviours, agents, 
2. Find the possible origin locations for the agents
3. Simulate occupancy behaviour and retrieve the cost of occupancy for each agent at each position
4. Generate the Cost matrix
5. Use the MIP Solver to optimise the combination to get the permutation matrix 


## Initilization

In [1]:
import os
import itertools
import sys
from math import factorial as fac
sys.path.append("D:/TU_Delft/Msc_Building_Technology/Semester_3/Graduation/Aditya_Graduation_Project_BT/06_Libraries")
import topogenesis as tg
import pyvista as pv
import trimesh as tm
import numpy as np
np.random.seed(0)
np.set_printoptions(threshold=sys.maxsize)
import networkx as nx
from itertools import combinations
import pickle

## Base Lattices

In [2]:
# loading the lattice from csv
lattice_path = os.path.relpath('Base_lattice_2.csv')
avail_lattice_base = tg.lattice_from_csv(lattice_path)
avail_lattice = avail_lattice_base*1
init_avail_lattice = tg.to_lattice(np.copy(avail_lattice*1), avail_lattice)

In [3]:
val_avail_lattice = tg.to_lattice(np.copy(avail_lattice*0), avail_lattice)+1

In [4]:
flattened_lattice = avail_lattice_base.flatten()

In [5]:
len(flattened_lattice)

1386

In [6]:
(np.sort(val_avail_lattice.centroids.T[1])).reshape(avail_lattice.shape)

cloud([[[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.],
        [ 3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.],
        [ 3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.],
        [ 3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.],
        [ 3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.],
        [ 3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.],
        [ 6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.],
        [ 6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.],
        [ 6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.],
        [ 6.,  6.,  6.,  

## Env Lattices

In [7]:
Bv = np.random.randint(1, 20, (np.shape(avail_lattice)), dtype='l')
Gv = np.random.randint(1, 20, (np.shape(avail_lattice)), dtype='l')
Yv = np.random.randint(1, 9, (np.shape(avail_lattice)), dtype='l')
Rv = np.random.randint(1, 9, (np.shape(avail_lattice)), dtype='l')
Vv = np.random.randint(1, 99, (np.shape(avail_lattice)), dtype='l')

## Stencils

In [8]:
# creating neighborhood definition
stencil_von_neumann = tg.create_stencil("von_neumann", 1, 1)
stencil_von_neumann.set_index([0,0,0], 0)
#print(stencil_von_neumann)

In [9]:
# creating neighborhood definition 
stencil_squareness = tg.create_stencil("moore", 1, 1)
# Reshaping the moore neighbourhood
stencil_squareness[0,:,:] = 0 
stencil_squareness[2,:,:] = 0
stencil_squareness.set_index([0,0,0], 0)
stencil_squareness_t = np.transpose(stencil_squareness) 
#print(stencil_squareness_t)

In [10]:
# creating neighborhood definition 
stencil_squareness_von = tg.create_stencil("von_neumann", 1, 1)
# Reshaping the moore neighbourhood
stencil_squareness_von[0,:,:] = 0 
stencil_squareness_von[2,:,:] = 0
stencil_squareness_von.set_index([0,0,0], 0)
stencil_squareness_von_t = np.transpose(stencil_squareness_von) 
#print(stencil_squareness_von)

In [11]:
stencil_cuboid = tg.create_stencil("moore", 1, 1)
stencil_cuboid.set_index([0,0,0], 0)
#print(stencil_cuboid)

In [12]:
Stencil_Find_neighbours_cuboid = tg.create_stencil("moore", 1, 1)

## Deriving all possible agent Center points

In [13]:
available_index_list= list(np.argwhere(flattened_lattice==1).flatten())

In [14]:
len(available_index_list)

684

In [15]:
Selected_positions=[]
for item in range(18,684,38):
    Selected_positions.append(available_index_list[item]) 

In [16]:
Selected_positions

[265,
 311,
 357,
 403,
 449,
 519,
 565,
 611,
 657,
 727,
 773,
 819,
 865,
 911,
 981,
 1027,
 1073,
 1119]

In [17]:
selected_neighbours= avail_lattice_base.find_neighbours(Stencil_Find_neighbours_cuboid)

In [18]:
#np.array(np.unravel_index(Selected_positions,init_avail_lattice.shape)).T

In [19]:
def all_possible_origins(flat_lattice,number_of_points_needed):
    reshaped_lattice= flat_lattice.reshape(99,4,16)*1
    reshaped_lattice_aggregated_value =np.sum(reshaped_lattice,axis=2)
    array = reshaped_lattice_aggregated_value.flatten()
    temp = array.argsort()
    ranks = np.empty_like(temp)
    ranks[temp] = np.arange(len(array))
    
    indices =list(range(6,flat_lattice.size,16))
    points_1D=[]
    for points in range(number_of_points_needed):
        index= np.where(ranks==points)
        points_1D.append(indices[int(index[0])] )  
    return points_1D


In [20]:
## The number of voxels which each agent has to occupy There are five agents :[Blue,Green,Yellow,Red,Violet]
## The number of voxels which each agent occupies are [B1,G1,Y1,R1,V1] 
mass_size = np.count_nonzero((avail_lattice==1)) 
Bn = 415 # Blue
Gn= 301 # Green
Yn=403 # Yellow
Rn= 19 # Red
Vn= 73 # Violet


In [21]:
#Divide the available lattice into grids to generate the necessary locations for origins??

In [22]:
def ranking_origins(point_id,value_lattice,origin_number):
    selected_neighbours= avail_lattice_base.find_neighbours(Stencil_Find_neighbours_cuboid)[point_id]
    neighbour_vals_for_agent= value_lattice.flatten()[selected_neighbours]
    neighbour_vals_aggregated_for_agent =np.sum(neighbour_vals_for_agent,axis=1)
    all_points = np.array(point_id)
    
    array = neighbour_vals_aggregated_for_agent
    temp = array.argsort()
    ranks = np.empty_like(temp)
    ranks[temp] = np.arange(len(array))
    
    a_lattice = ranks.flatten()
    Co_ordinates = []
    for item in range(0,origin_number):
        Co_ordinates.append(all_points[np.argwhere(a_lattice==item)].flatten()[0])    
        
    return Co_ordinates

In [23]:
# agent class
class agent():
    def __init__(self, origin, stencil, id):

        # define the origin attribute of the agent and making sure that it is an intiger
        self.origin = np.array(origin).astype(int)
        # define old origin attribute and assigning the origin to it as the initial state
        self.old_origin = self.origin
        # define stencil of the agent
        self.stencil = stencil
        #define agent id
        self.id = id

    # definition of random/argmax occupancy on a 2d squarish stencil 
    def random_occupy_squareness(self, env):
        # retrieve the list of neighbours of the agent based on the stencil
        neighs = env.availibility.find_neighbours_masked(self.stencil, loc = self.origin)
        neighs_full_floor = env.availibility.find_neighbours_masked(stencil_full_floor, loc = self.origin)
        # find availability of neighbours
        neighs_availibility = env.availibility.flatten()[neighs]
        neighs_availibility_full_floor = env.availibility.flatten()[neighs_full_floor]
        # separate available neighbours
        free_neighs = neighs[neighs_availibility==1]
        free_neighs_full_floor = neighs_full_floor[neighs_availibility_full_floor==1]
        #print(free_neighs)
        if len(free_neighs)== 0 :
            free_neighs = free_neighs_full_floor
        else: 
            free_neighs= free_neighs
        # retrieve the value of each neighbour
        free_neighs_value = env.value.flatten()[free_neighs]
        # find the neighbour with maximum my value
       # selected_neigh = free_neighs[np.argmax(free_neighs_value)]
        selected_neigh = np.random.choice(free_neighs,1)
        #print(selected_neigh)
        # update information
        ####################
        # set the current origin as the ol origin
        self.old_origin = self.origin
        # update the current origin with the new selected neighbour
        self.origin = np.array(np.unravel_index(selected_neigh, env.availibility.shape)).flatten()
        #print(self.origin)
     
      # definition of random/argmax occupancy on a 3d cubish stencil
    def random_occupy_cubish(self, env):
        # retrieve the list of neighbours of the agent based on the stencil
        neighs = env.availibility.find_neighbours_masked(self.stencil, loc = self.origin)
        neighs_full_lattice = env.availibility.find_neighbours_masked(stencil_full_lattice, loc = self.origin)
        # find availability of neighbours
        neighs_availibility = env.availibility.flatten()[neighs]
        neighs_availibility_full_lattice = env.availibility.flatten()[neighs_full_lattice]
        # separate available neighbours
        free_neighs = neighs[neighs_availibility==1]
        free_neighs_full_lattice = neighs_full_lattice[neighs_availibility_full_lattice==1]
        #print(free_neighs)
        if len(free_neighs)== 0 :
            free_neighs = free_neighs_full_lattice
        else: 
            free_neighs= free_neighs
        # retrieve the value of each neighbour
        free_neighs_value = env.value.flatten()[free_neighs]
        # find the neighbour with maximum my value
        selected_neigh = free_neighs[np.argmax(free_neighs_value)]
        #selected_neigh = np.random.choice(free_neighs,1)
        #print(selected_neigh)
        # update information
        ####################
        # set the current origin as the ol origin
        self.old_origin = self.origin
        # update the current origin with the new selected neighbour
        self.origin = np.array(np.unravel_index(selected_neigh, env.availibility.shape)).flatten()
        #print(self.origin)
      
        
    def random_occupy_cubish_von_neumann(self, env):
        # retrieve the list of neighbours of the agent based on the stencil
        neighs = env.availibility.find_neighbours_masked(self.stencil, loc = self.origin)
        neighs_full_lattice = env.availibility.find_neighbours_masked(stencil_cuboid, loc = self.origin)
        # find availability of neighbours
        neighs_availibility = env.availibility.flatten()[neighs]
        neighs_availibility_full_lattice = env.availibility.flatten()[neighs_full_lattice]
        # separate available neighbours
        free_neighs = neighs[neighs_availibility==1]
        free_neighs_full_lattice = neighs_full_lattice[neighs_availibility_full_lattice==1]
        #print(free_neighs)
        if len(free_neighs)== 0 :
            free_neighs = free_neighs_full_lattice
        else: 
            free_neighs= free_neighs
        # retrieve the value of each neighbour
        free_neighs_value = env.value.flatten()[free_neighs]
        # find the neighbour with maximum my value
        selected_neigh = np.random.choice(free_neighs,1)
        #print(selected_neigh)
        # update information
        ####################
        # set the current origin as the ol origin
        self.old_origin = self.origin
        # update the current origin with the new selected neighbour
        self.origin = np.array(np.unravel_index(selected_neigh, env.availibility.shape)).flatten()
        #print(self.origin)
        
    def argmax_occupy_von_neumann(self, env):
        # retrieve the list of neighbours of the agent based on the stencil
        neighs = env.availibility.find_neighbours_masked(self.stencil, loc = self.origin)
        neighs_full_lattice = env.availibility.find_neighbours_masked(stencil_full_lattice, loc = self.origin)
        # find availability of neighbours
        neighs_availibility = env.availibility.flatten()[neighs]
        neighs_availibility_full_lattice = env.availibility.flatten()[neighs_full_lattice]
        # separate available neighbours
        free_neighs = neighs[neighs_availibility==1]
        free_neighs_full_lattice = neighs_full_lattice[neighs_availibility_full_lattice==1]
        #print(free_neighs)
        if len(free_neighs)== 0 :
            free_neighs = free_neighs_full_lattice
        else: 
            free_neighs= free_neighs
        # retrieve the value of each neighbour
        free_neighs_value = env.value.flatten()[free_neighs]
        # find the neighbour with maximum my value
        selected_neigh = free_neighs[np.argmax(free_neighs_value)]
        #selected_neigh = np.random.choice(free_neighs,1)
        #print(selected_neigh)
        # update information
        ####################
        # set the current origin as the ol origin
        self.old_origin = self.origin
        # update the current origin with the new selected neighbour
        self.origin = np.array(np.unravel_index(selected_neigh, env.availibility.shape)).flatten()
        #print(self.origin)

        
     # definition of 2d occupying method for agents
    def one_neighbour_occupy_squareness_moore(self, env):
        # retrieve the list of neighbours of the agent based on the stencil
        neighs = env.availibility.find_neighbours_masked(self.stencil, loc = self.origin)
        #print(neighs)
        neighs_full_floor = env.availibility.find_neighbours_masked(stencil_full_floor, loc = self.origin)

        # find availability of neighbours
        neighs_availibility = env.availibility.flatten()[neighs]               
        neighs_availibility_full_floor = env.availibility.flatten()[neighs_full_floor]
        #print(neighs_availibility)
        
        # find env values of all neighbours
        all_neighs_value = env.value.flatten()[neighs]
        all_neighs_value_mod = np.copy(all_neighs_value)
        
        
        #finding number of neighbours and bumping the values based on adjacency for a 9 neighbourhood
        
        #print(neighbourhood_details)
        one = neighs_availibility[1] + neighs_availibility[2] 
        two = neighs_availibility[0] + neighs_availibility[2] 
        three = neighs_availibility[1] + neighs_availibility[4] 
        four = neighs_availibility[0] + neighs_availibility[6] 
        five = neighs_availibility[2] + neighs_availibility[7] 
        six = neighs_availibility[3] + neighs_availibility[6] 
        seven = neighs_availibility[5] + neighs_availibility[7] 
        eight = neighs_availibility[6] + neighs_availibility[4] 
        neighbourhood_details = [one,two,three,four,five,six,seven,eight]
        
        #print(neighbourhood_details)
        for detail in range(len(neighs_availibility)-1):
            neighbourhood_condition = neighbourhood_details[detail] 
            #print(neighbourhood_condition)
            if neighbourhood_condition == 3:
                all_neighs_value_mod[detail]= all_neighs_value_mod[detail] + one_neighbour_factor
            elif neighbourhood_condition == 4:
                all_neighs_value_mod[detail]= all_neighs_value_mod[detail] + two_neighbour_factor
            else:
                all_neighs_value_mod[detail] = all_neighs_value_mod[detail]
        #print(all_neighs_value_mod)   
        

        neighs_value_flattened = env.value.flatten()
        for val_mod in all_neighs_value_mod:
            for neigh in neighs :
                neighs_value_flattened[neigh]=val_mod
        
        
        # separate available neighbours
        free_neighs = neighs[neighs_availibility==1]
        free_neighs_full_floor = neighs_full_floor[neighs_availibility_full_floor==1]
        #print(free_neighs)
        if len(free_neighs)== 0 :
            free_neighs = free_neighs_full_floor
        else: 
            free_neighs= free_neighs
        # retrieve the value of each neighbour
        free_neighs_value = neighs_value_flattened[free_neighs]
        
        #print(free_neighs_value)
        # find the neighbour with maximum my value
        selected_neigh = free_neighs[np.argmax(free_neighs_value)]
        #print(selected_neigh)
        # update information
        ####################
        # set the current origin as the ol origin
        self.old_origin = self.origin
        # update the current origin with the new selected neighbour
        self.origin = np.array(np.unravel_index(selected_neigh, env.availibility.shape)).flatten()
        #print(self.origin)
    
    
         # definition of 2d occupying method for agents
    def one_neighbour_occupy_squareness_von_neumann(self, env):
        
         # retrieve the list of neighbours of the agent based on the stencil
        neighs = env.availibility.find_neighbours_masked(self.stencil, loc = self.origin)
        neighs_full_floor = env.availibility.find_neighbours_masked(stencil_full_lattice, loc = self.origin)
        # find availability of neighbours
        neighs_availibility = env.availibility.flatten()[neighs]
        neighs_availibility_full_floor = env.availibility.flatten()[neighs_full_floor]
        # separate available neighbours
        free_neighs = neighs[neighs_availibility==1]
        free_neighs_full_floor = neighs_full_floor[neighs_availibility_full_floor==1]
        #print(free_neighs)
        if len(free_neighs)== 0 :
            free_neighs = free_neighs_full_floor
        else: 
            free_neighs= free_neighs
        # retrieve the value of each neighbour
        free_neighs_value = env.value.flatten()[free_neighs]
        # find the neighbour with maximum my value
       # selected_neigh = free_neighs[np.argmax(free_neighs_value)]
        selected_neigh = np.random.choice(free_neighs,1)
        #print(selected_neigh)
        # update information
        ####################
        # set the current origin as the ol origin
        self.old_origin = self.origin
        # update the current origin with the new selected neighbour
        self.origin = np.array(np.unravel_index(selected_neigh, env.availibility.shape)).flatten()
        #print(self.origin)
    def one_neighbour_occupy_squareness_behaviour (self,env):        
        value_lattice_flat = env.value.flatten()
        sqr_factor = 10.1
        sqr_shift = 10.0
        init_loc = self.origin
        neighs_full_lattice = env.availibility.find_neighbours_masked(stencil_full_lattice, loc = self.origin)
        neighs_availibility_full_lattice = env.availibility.flatten()[neighs_full_lattice]
        free_neighs_full_lattice = neighs_full_lattice[neighs_availibility_full_lattice==1]
        
        agn_locs = [list(init_loc)]
        all_neighs =[] 
        avail_lattice_flat = env.availibility.flatten()
        neighs = env.availibility.find_neighbours_masked(self.stencil, loc = self.origin)
        all_neighs.append(neighs)
        env.neigh_squareness.append(neighs)

        neighs_flatten = np.array(env.neigh_squareness).flatten()
        #print(neighs_flatten)
        neighs_availability = avail_lattice_flat[neighs_flatten]
  
        # keep the available ones only
        avail_neighs = neighs_flatten[neighs_availability==1] 
        
        if len(avail_neighs)== 0 :
            avail_neighs = free_neighs_full_lattice
        else: 
            avail_neighs= avail_neighs
        #print(avail_neighs)
        avail_unq_neighs, avail_unq_neighs_count = np.unique(avail_neighs, return_counts=True)
        #print(avail_unq_neighs)
        #print(avail_unq_neighs_count)
        neighs_unq_base_value = value_lattice_flat[avail_unq_neighs]
        neigh_sqr_evaluation = np.power(sqr_factor, (avail_unq_neighs_count - 1)) * neighs_unq_base_value + sqr_shift
        #neigh_sqr_evaluation = neighs_unq_base_value + sqr_shift * (avail_unq_neighs_count - 1)


        selected_neigh_index = np.argmax(neigh_sqr_evaluation)
        selected_neigh_1D_id = avail_unq_neighs[selected_neigh_index]
        #selected_neigh_3D_id = np.unravel_index(selected_neigh_1D_id,bounds.shape )

        # update information
        ####################
        self.old_origin = self.origin
        # update the current origin with the new selected neighbour
        self.origin = np.array(np.unravel_index(selected_neigh_1D_id, env.availibility.shape)).flatten()
   
        
        
    def one_neighbour_occupy_cubish_behaviour (self,env):        
        value_lattice_flat = env.value.flatten()
        sqr_factor = 10.1
        sqr_shift = 10.0
        init_loc = self.origin
        neighs_availibility_full_lattice = env.availibility.flatten()
        free_neighs_full_lattice = np.argwhere(neighs_availibility_full_lattice==1).flatten()
        
        agn_locs = [list(init_loc)]
        all_neighs =[] 
        avail_lattice_flat = env.availibility.flatten()
        neighs = env.availibility.find_neighbours_masked(self.stencil, loc = self.origin)
        all_neighs.append(neighs)
        env.neigh_cubish.append(neighs)

        neighs_flatten = np.array(env.neigh_cubish).flatten()
        #print(neighs_flatten)
        neighs_availability = avail_lattice_flat[neighs_flatten]
  
        # keep the available ones only
        avail_neighs = neighs_flatten[neighs_availability==1] 
        
        if len(avail_neighs)== 0 :
            avail_neighs = free_neighs_full_lattice
        else: 
            avail_neighs= avail_neighs
                
        #print(avail_neighs)
        avail_unq_neighs, avail_unq_neighs_count = np.unique(avail_neighs, return_counts=True)
        #print(avail_unq_neighs)
        #print(avail_unq_neighs_count)
        neighs_unq_base_value = value_lattice_flat[avail_unq_neighs]
        
        #neigh_sqr_evaluation = np.power(sqr_factor, (avail_unq_neighs_count - 1)) * neighs_unq_base_value + sqr_shift
        neigh_sqr_evaluation = neighs_unq_base_value + sqr_shift * (avail_unq_neighs_count - 1)

       # print(neighs_unq_base_value) 
        selected_neigh_index = np.argmax(neigh_sqr_evaluation)
        selected_neigh_1D_id = avail_unq_neighs[selected_neigh_index]
        #selected_neigh_3D_id = np.unravel_index(selected_neigh_1D_id,bounds.shape )       
            
        # update information
        ####################
        self.old_origin = self.origin
        # update the current origin with the new selected neighbour
        self.origin = np.array(np.unravel_index(selected_neigh_1D_id, env.availibility.shape)).flatten()
        #print(selected_neigh_1D_id)
        #print(free_neighs_full_lattice)

            

In [24]:
# Agent init class

def initialize_agents_random_origin (stencil,avail_lattice):
    #finding origin 
    agn_num = 1
    occ_lattice = avail_lattice*0 -1
    avail_flat = avail_lattice.flatten()
    avail_index = np.array(np.where(avail_lattice == 1)).T
    select_id = np.random.choice(len(avail_index), agn_num)
    agn_origins = tuple(avail_index[select_id].flatten()) 

    # Defining agents
    myagent = agent(agn_origins, stencil, select_id)

    return myagent

def initialize_agents_fixed_origin (stencil,avail_lattice,origin):
    #finding origin 
    agn_origins = np.unravel_index(origin,avail_lattice.shape)
    select_id = origin
    # Defining agents
    myagent = agent(agn_origins, stencil, select_id)

    return myagent

In [25]:
# environment class
class environment():
    def __init__(self, lattices, agents,number_of_iterations,method_name):
        self.availibility = lattices["availibility"]
        self.value = lattices["enviornment"]
        self.agent_origin = self.availibility 
        self.agents = agents
        self.update_agents()
        self.number_of_iterations = number_of_iterations
        self.method_name = method_name
        self.neigh_cubish = []
        self.neigh_squareness = []
    def update_agents(self):
        # making previous position available
      #  self.availibility[tuple(self.agents.old_origin)] = self.availibility[tuple(self.agents.old_origin)] * 0 + 1
        # removing agent from previous position
        self.agent_origin[tuple(self.agents.old_origin)] *= 0+1
        # making the current position unavailable
        self.availibility[tuple(self.agents.origin)] = self.agents.id
        # adding agent to the new position 
        self.agent_origin[tuple(self.agents.origin)] = self.agents.id
    
    def random_occupy_squareness_agents(self):
        # iterate over egents and perform the walk
        self.agents.random_occupy_squareness(self)
        # update the agent states in environment
        self.update_agents()
        
    def random_occupy_cubish_agents(self):
        # iterate over egents and perform the walk
        self.agents.random_occupy_cubish(self)
        # update the agent states in environment
        self.update_agents()
    
    def random_occupy_cubish_von_neumann_agents(self):
        # iterate over egents and perform the walk
        self.agents.random_occupy_cubish_von_neumann(self)
        # update the agent states in environment
        self.update_agents()
    
    def argmax_occupy_von_neumann(self):
        # iterate over egents and perform the walk
        self.agents.argmax_occupy_von_neumann(self)
        # update the agent states in environment
        self.update_agents()
        
    def one_neighbour_occupy_squareness_moore(self):
        # iterate over egents and perform the walk
        self.agents.one_neighbour_occupy_squareness_moore(self)
        # update the agent states in environment
        self.update_agents()
        
    def one_neighbour_occupy_squareness_von_neumann(self):
        # iterate over egents and perform the walk
        self.agents.one_neighbour_occupy_squareness_von_neumann(self)
        # update the agent states in environment
        self.update_agents()
      
    def one_neighbour_occupy_cubish_behaviour(self):
        # iterate over egents and perform the walk
        self.agents.one_neighbour_occupy_cubish_behaviour(self)
        # update the agent states in environment
        self.update_agents()
        
    def one_neighbour_occupy_squareness_behaviour(self):
        # iterate over egents and perform the walk
        self.agents.one_neighbour_occupy_squareness_behaviour(self)
        # update the agent states in environment
        self.update_agents()
        

In [26]:
def calculate_cost_for_positions(Location_list,Enviornment_lattice,number_of_cells_to_be_occupied):
    All_viz_options = []
    for indexing, item in enumerate(Location_list):
        #print(indexing)
        Agent = agent(np.unravel_index(item,init_avail_lattice.shape),stencil_cuboid,9)

        occ_lattice_sim = tg.to_lattice(np.copy(avail_lattice), avail_lattice)

        env = {"availibility": occ_lattice_sim,"enviornment": Enviornment_lattice}


        env_1 = environment(env,Agent,number_of_cells_to_be_occupied,"one_neighbour_occupy_cubish_behaviour")

        env_availability_viz = []
        env_list =[env_1]
        number_steps = max(map(lambda e:e.number_of_iterations,env_list))    
        for a in range(number_steps):
            # print(env.availibility)
            #print(env.agent_origin)

            for e in env_list:
                if a < e.number_of_iterations :
                    #print(a)
                    #print(e.number_of_iterations)
                    if e.method_name == "one_neighbour_occupy_squareness_moore":
                        e.one_neighbour_occupy_squareness_moore()

                    elif e.method_name == "one_neighbour_occupy_cubish_agents" :
                        e.one_neighbour_occupy_cubish_agents()

                    elif e.method_name == "random_occupy_squareness_agents" :
                        e.random_occupy_squareness_agents()

                    elif e.method_name == "random_occupy_cubish_agents" :
                        e.random_occupy_cubish_agents()  

                    elif e.method_name == "random_occupy_cubish_von_neumann_agents" :
                        e.random_occupy_cubish_von_neumann_agents()                           

                    elif e.method_name == "one_neighbour_occupy_squareness_von_neumann" :
                        e.one_neighbour_occupy_squareness_von_neumann()                


                    elif e.method_name == "one_neighbour_occupy_squareness_behaviour" :
                        e.one_neighbour_occupy_squareness_behaviour()  

                    elif e.method_name == "one_neighbour_occupy_cubish_behaviour" :
                        e.one_neighbour_occupy_cubish_behaviour()  

                    elif e.method_name == "argmax_occupy_von_neumann" :
                        e.argmax_occupy_von_neumann()  

        env_availability_viz.append(e.availibility)

        All_viz_options.append(env_availability_viz)

    all_viz_array= np.array(All_viz_options)
    all_viz_array_shape = all_viz_array.shape
    reshaped_all_viz= all_viz_array.reshape(all_viz_array_shape[0]*all_viz_array_shape[1],all_viz_array_shape[2],all_viz_array_shape[3],all_viz_array_shape[4])
    sum_of_values = []
    for lattice in reshaped_all_viz:
        indexes= np.argwhere(lattice.flatten()==9).flatten()
        values= env["enviornment"].flatten()[indexes]
        total = int(np.sum(values))
        sum_of_values.append(total)
    return sum_of_values

In [27]:
Cost_for_B = np.array(calculate_cost_for_positions(Selected_positions,Bv,45)) # Private Housing
Cost_for_G = np.array(calculate_cost_for_positions(Selected_positions,Gv,45)) # Social Housing
Cost_for_Y = np.array(calculate_cost_for_positions(Selected_positions,Yv,45)) # Free sector Housing
Cost_for_R = np.array(calculate_cost_for_positions(Selected_positions,Rv,45)) # Restaurant and Cafe 
Cost_for_V = np.array(calculate_cost_for_positions(Selected_positions,Vv,45)) # Retail stores


In [28]:
cost_matrix = np.concatenate ((np.tile(Cost_for_B,4),np.tile(Cost_for_G,4),np.tile(Cost_for_Y,3),np.tile(Cost_for_R,2),np.tile(Cost_for_V,5)))

In [29]:
cost_matrix_reshaped = cost_matrix.reshape (int(cost_matrix.size/18), 18 )

In [30]:
len(cost_matrix_reshaped)

18

In [31]:
from ortools.linear_solver import pywraplp
location_assignment =[]

def main():
    # Data from all the colours on all the origins
    costs = cost_matrix_reshaped
    num_workers = 18

    num_tasks = len(costs[0])
    #print(num_tasks)
    # Solver
    # Create the mip solver with the SCIP backend.
    solver = pywraplp.Solver.CreateSolver('SCIP')


    # Variables
    # x[i, j] is an array of 0-1 variables, which will be 1
    # if worker i is assigned to task j.
    x = {}
    for i in range(num_workers):
        for j in range(num_tasks):
            x[i, j] = solver.IntVar(0, 1, '')

    # Constraints
    # Each worker is assigned to at most 1 task.
    for i in range(num_workers):
        solver.Add(solver.Sum([x[i, j] for j in range(num_tasks)]) <= 1)

    # Each task is assigned to exactly one worker.
    for j in range(num_tasks):
        solver.Add(solver.Sum([x[i, j] for i in range(num_workers)]) == 1)

    # Objective
    objective_terms = []
    for i in range(num_workers):
        for j in range(num_tasks):
            objective_terms.append(costs[i][j] * x[i, j])
    solver.Maximize(solver.Sum(objective_terms))

    # Solve
    status = solver.Solve()

    # Print solution.
    if status == pywraplp.Solver.OPTIMAL or status == pywraplp.Solver.FEASIBLE:
        print('Total cost = ', solver.Objective().Value(), '\n')
        for i in range(num_workers):
            for j in range(num_tasks):
                # Test if x[i,j] is 1 (with tolerance for floating point arithmetic).
                if x[i, j].solution_value() > 0.5:
                    print('Agent %d assigned to Location %d.  Cost = %d' %
                          (i, j, costs[i][j]))
                    location_assignment.append(j)


if __name__ == '__main__':
    main()
    


Total cost =  21289.0 

Agent 0 assigned to Location 11.  Cost = 587
Agent 1 assigned to Location 6.  Cost = 544
Agent 2 assigned to Location 13.  Cost = 529
Agent 3 assigned to Location 2.  Cost = 565
Agent 4 assigned to Location 10.  Cost = 514
Agent 5 assigned to Location 7.  Cost = 554
Agent 6 assigned to Location 15.  Cost = 523
Agent 7 assigned to Location 1.  Cost = 514
Agent 8 assigned to Location 5.  Cost = 221
Agent 9 assigned to Location 4.  Cost = 213
Agent 10 assigned to Location 0.  Cost = 199
Agent 11 assigned to Location 9.  Cost = 231
Agent 12 assigned to Location 14.  Cost = 231
Agent 13 assigned to Location 8.  Cost = 3154
Agent 14 assigned to Location 17.  Cost = 3154
Agent 15 assigned to Location 16.  Cost = 3223
Agent 16 assigned to Location 12.  Cost = 3160
Agent 17 assigned to Location 3.  Cost = 3173


In [32]:
Agent_origins = []
for index in location_assignment:
    Agent_origins.append(Selected_positions[index])

In [33]:
def agent_based_on_origin(Origin,index):
    return agent(np.unravel_index(Origin,init_avail_lattice.shape),stencil_cuboid,index)

In [34]:
occ_lattice_sim = tg.to_lattice(np.copy(avail_lattice), avail_lattice)
env_B = {"availibility": avail_lattice,"enviornment": Bv}
env_G = {"availibility": avail_lattice,"enviornment": Gv}
env_Y = {"availibility": avail_lattice,"enviornment": Yv}
env_R = {"availibility": avail_lattice,"enviornment": Rv}
env_V = {"availibility": avail_lattice,"enviornment": Vv}


In [35]:
Origins_B = Agent_origins[:4]
print(Origins_B)
Agents_B =[]
env_1 =[]
for i,item in enumerate(Origins_B):
    created_agent_1= agent_based_on_origin(item,10)
    Agents_B.append(created_agent_1)
    env_1.append(environment(env_B,created_agent_1,33,"one_neighbour_occupy_cubish_behaviour"))

[819, 565, 911, 357]


In [36]:
Origins_G = Agent_origins[4:8]
print(Origins_G)
Agents_G =[]
env_2 =[]
for item in Origins_G:
    created_agent_2 = agent_based_on_origin(item,100)
    Agents_G.append(created_agent_2)
    env_2.append(environment(env_G,created_agent_2,33,"one_neighbour_occupy_cubish_behaviour"))

[773, 611, 1027, 311]


In [37]:
Origins_Y = Agent_origins[8:11]
print(Origins_Y)
Agents_Y =[]
env_3 =[]
for item in Origins_Y:
    created_agent_3 = agent_based_on_origin(item,200)
    Agents_Y.append(created_agent_3)
    env_3.append(environment(env_Y,created_agent_3,29,"one_neighbour_occupy_cubish_behaviour"))

[519, 449, 265]


In [38]:
Origins_R = Agent_origins[11:13]
print(Origins_R)
Agents_R =[]
env_4 =[]
for item in Origins_R:
    created_agent_4 = agent_based_on_origin(item,300)
    Agents_R.append(created_agent_4)
    env_4.append(environment(env_R,created_agent_4,44,"one_neighbour_occupy_cubish_behaviour"))

[727, 981]


In [39]:
Origins_V = Agent_origins[13:18]
print(Origins_V)
Agents_V =[]
env_5 =[]
for item in Origins_V:
    created_agent_5 = agent_based_on_origin(item,400)
    Agents_V.append(created_agent_5)
    env_5.append(environment(env_V,created_agent_5,45,"one_neighbour_occupy_cubish_behaviour"))

[657, 1119, 1073, 865, 403]


In [40]:
all_enviornments= np.concatenate(((np.array(env_1)),(np.array(env_2)),(np.array(env_3)),(np.array(env_4)),(np.array(env_5))))


In [41]:
env_availability_viz = []
env_list = all_enviornments
number_steps = max(map(lambda e:e.number_of_iterations,env_list))
for a in range(number_steps):
    # print(env.availibility)
    #print(env.agent_origin)
    print(a)
    for e in env_list:
        if a < e.number_of_iterations :
            #print(a)
            #print(e.number_of_iterations)
            if e.method_name == "one_neighbour_occupy_squareness_moore":
                e.one_neighbour_occupy_squareness_moore()
                
            elif e.method_name == "one_neighbour_occupy_cubish_agents" :
                e.one_neighbour_occupy_cubish_agents()
                
            elif e.method_name == "random_occupy_squareness_agents" :
                e.random_occupy_squareness_agents()
            
            elif e.method_name == "random_occupy_cubish_agents" :
                e.random_occupy_cubish_agents()  
                
            elif e.method_name == "random_occupy_cubish_von_neumann_agents" :
                e.random_occupy_cubish_von_neumann_agents()                           
                
            elif e.method_name == "one_neighbour_occupy_squareness_von_neumann" :
                e.one_neighbour_occupy_squareness_von_neumann()                
                
                
            elif e.method_name == "one_neighbour_occupy_squareness_behaviour" :
                e.one_neighbour_occupy_squareness_behaviour()  
                
            elif e.method_name == "one_neighbour_occupy_cubish_behaviour" :
                e.one_neighbour_occupy_cubish_behaviour()  
            
            elif e.method_name == "argmax_occupy_von_neumann" :
                e.argmax_occupy_von_neumann()  
                
            env_availability_viz.append(e.availibility-1)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44


In [42]:
env_availability_viz[-1]

lattice([[[ -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1],
          [ -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1],
          [ -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1],
          [ -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1],
          [ -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1],
          [ -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1],
          [ -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1],
          [ -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1],
          [ -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1],
          [ -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1],
          [ -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1],
          [ -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1],
          [ -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1],
          [ -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1],
          [ -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  

In [43]:
len(np.argwhere(env_availability_viz[-1]==0))

2

In [44]:
p = pv.Plotter(notebook=True)

base_lattice = env_availability_viz[0]
print(base_lattice.unit)
# Set the grid dimensions: shape + 1 because we want to inject our values on the CELL data
grid = pv.UniformGrid()
grid.dimensions = np.array(base_lattice.shape) + 1
# The bottom left corner of the data set
grid.origin = base_lattice.minbound - base_lattice.unit * 0.5
# These are the cell sizes along each axis
grid.spacing = base_lattice.unit 

# adding the boundingbox wireframe
p.add_mesh(grid.outline(), color="grey", label="Domain")

# adding the avilability lattice
init_avail_lattice.fast_vis(p)

# adding axes
p.add_axes()
p.show_bounds(grid="back", location="back", color="#aaaaaa") 

def create_mesh(value):
    f = int(value)
    lattice = env_availability_viz[f]

    # Add the data values to the cell data
    grid.cell_arrays["Agents"] = lattice.flatten(order="F").astype(int)  # Flatten the array!
    # filtering the voxels
    threshed = grid.threshold([1.1, avail_lattice.size])
    # adding the voxels
    p.add_mesh(threshed, name='sphere', show_edges=True, opacity=1.0, show_scalar_bar=False)
    return
number_steps_2 = 684
p.add_slider_widget(create_mesh, [0, number_steps_2], title='Time', value=0, event_type="always", style="classic")


p.show(use_ipyvtk=True)


[3 3 3]


ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(89.32480766111564, 111.82480766111564, 96.82480766111564),
 (7.5, 30.0, 15.0),
 (0.0, 0.0, 1.0)]

In [45]:
BC_index = np.argwhere(avail_lattice.flatten()==10).flatten()
GC_index = np.argwhere(avail_lattice.flatten()==100).flatten()
YC_index = np.argwhere(avail_lattice.flatten()==200).flatten()
RC_index = np.argwhere(avail_lattice.flatten()==300).flatten()
VC_index = np.argwhere(avail_lattice.flatten()==400).flatten()

In [46]:
BC= np.sum(Bv.flatten()[BC_index])
GC= np.sum(Gv.flatten()[GC_index])
YC= np.sum(Yv.flatten()[YC_index])
RC= np.sum(Rv.flatten()[RC_index])
VC= np.sum(Vv.flatten()[VC_index])
total_cost = BC + GC + YC+ RC +  VC

In [47]:
BC

1394

In [48]:
"""
for i, lattice in enumerate(env_availability_viz):
    csv_path = os.path.relpath('csv_agent_rush/abm_f_'+ f'{i:03}' + '.csv')
    lattice.to_csv(csv_path)
"""

"\nfor i, lattice in enumerate(env_availability_viz):\n    csv_path = os.path.relpath('csv_agent_rush/abm_f_'+ f'{i:03}' + '.csv')\n    lattice.to_csv(csv_path)\n"